In [11]:
import boto3
from IPython.display import clear_output, Image, display, HTML
import numpy as np
import cv2
import base64
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, push_notebook
import time
import json
output_notebook()

Loading BokehJS ...

In [2]:
STREAM_NAME = "pi4-001"
kvs = boto3.client("kinesisvideo")
# Grab the endpoint from GetDataEndpoint
endpoint = kvs.get_data_endpoint(
    APIName="GET_HLS_STREAMING_SESSION_URL",
    StreamName=STREAM_NAME
)['DataEndpoint']

print(endpoint)

https://b-647daf39.kinesisvideo.us-west-2.amazonaws.com


In [3]:
class VideoPlayer(object):
    def __init__(self):
        self._init = False
        self._myImage = None
        
    def __call__(self, frame):
        if frame is None:
            return
        if self._init is False:
            self.init_display(frame)
            self._init = True
        else:
            self.update_display(frame)

    def init_display(self, frame):
        assert frame is not None
        frame=cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA) # because Bokeh expects a RGBA image
        frame=cv2.flip(frame, -1) # because Bokeh flips vertically
        width=frame.shape[1]
        height=frame.shape[0]
        p = figure(x_range=(0,width), y_range=(0,height), output_backend="webgl", width=width, height=height)
        self._myImage = p.image_rgba(image=[frame], x=0, y=0, dw=width, dh=height)
        show(p, notebook_handle=True)
    
    def update_display(self, frame):
        assert frame is not None
        frame=cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
        frame=cv2.flip(frame, -1)
        self._myImage.data_source.data['image']=[frame]
        push_notebook()

In [4]:
# # Grab the HLS Stream URL from the endpoint
kvam = boto3.client("kinesis-video-archived-media", endpoint_url=endpoint)
url = kvam.get_hls_streaming_session_url(
    StreamName=STREAM_NAME,
    PlaybackMode="LIVE"
)['HLSStreamingSessionURL']
vcap = cv2.VideoCapture(url)
player = VideoPlayer()
ret, frame = vcap.read()

In [9]:
import sagemaker
sagemaker_endpoint_name = 'object-detection-2019-10-18-17-56-41-925'
sagemaker_endpoint = sagemaker.predictor.RealTimePredictor(sagemaker_endpoint_name)
sagemaker_endpoint.content_type = 'image/jpeg'

In [14]:
while(True):
    # Capture frame-by-frame
    ret, frame = vcap.read()

    if frame is not None:
        # Display the resulting frame
        #cv2.imshow('frame',frame)
        player(frame)
        time.sleep(0.1)
        
        filename = 'frame_'+str(time.time())+'.jpg'
        cv2.imwrite(filename, frame)
        with open(filename, 'rb') as image:
            f = image.read()
            b = bytearray(f)
        result = sagemaker_endpoint.predict(b)
        print(json.loads(result))
        
        #break

        # Press q to close the video windows before it ends if you want
        #if cv2.waitKey(22) & 0xFF == ord('q'):
        #    break
    else:
        print("Frame is None")
        break

# When everything done, release the capture
vcap.release()
print("Video stop")

{'prediction': [[0.0, 0.24752621352672577, 0.42073118686676025, 0.16536372900009155, 0.8113102912902832, 1.0], [0.0, 0.22004914283752441, 0.18011510372161865, 0.18545567989349365, 0.5559449195861816, 1.0], [0.0, 0.14285284280776978, 0.0, 0.18450778722763062, 0.3047534227371216, 1.0], [0.0, 0.13757559657096863, 0.7061977386474609, 0.0, 0.9481282234191895, 0.487520307302475], [0.0, 0.1356004774570465, 0.4443764090538025, 0.026761233806610107, 0.6864836812019348, 0.592413067817688], [0.0, 0.13361965119838715, 0.5761809945106506, 0.0, 0.8101549744606018, 0.474985271692276], [0.0, 0.13080069422721863, 0.19671504199504852, 0.2754206359386444, 0.4383842945098877, 0.8326259851455688], [0.0, 0.1270255595445633, 0.19511312246322632, 0.04157248139381409, 0.4365425705909729, 0.5913702249526978], [0.0, 0.12628869712352753, 0.4450029730796814, 0.2912696599960327, 0.6884269118309021, 0.8445420265197754], [0.0, 0.1255466789007187, 0.3256353735923767, 0.28016945719718933, 0.5614745616912842, 0.83609318

KeyboardInterrupt: 